## Requirements

In [54]:
# # Upgrade pip
# python -m pip install --upgrade pip

# # Install essential Python packages
# pip install -U ipykernel seaborn matplotlib pandas numpy plotly nbformat altair plotnine scikit-learn 


## Dataset Overview

**Source:**  
This dataset is sourced from Kaggle: [An Online Shop Business](https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business/data).

### Business Context

- **Location & History:**  
  The dataset comes from a UK-based online retail store that sells gifts and homewares. The business has been operating since 2007.

- **Customer Base:**  
  The store caters to individual customers worldwide as well as small businesses that purchase in bulk for resale.

- **Sales Channels:**  
  Most transactions are direct purchases made through the website.

### Dataset Details

- **Size:**  
  500K rows and 8 columns.

- **Columns Include:**  
  - Customer number  
  - Transaction number  
  - Product number  
  - Product name  
  - Price per unit (in pound sterling £)  
  - Quantity  
  - Date  

- **Note on Cancellations:**  
  A small percentage of orders are cancelled, primarily due to out-of-stock conditions. Customers often cancel orders when they prefer to receive all products together.

### Purpose of Analysis

This dataset is ideal for analyzing:
- **Sales Performance:** Total revenue, average order value, etc.
- **Product Demand and Cancellation Trends:** Identifying which products drive sales and which have high cancellation rates.
- **Customer Behavior:** Purchase frequency and spending patterns.


## Importing Required Libraries

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Load Dataset

In [56]:
# Load the dataset from a CSV file
df=pd.read_csv('Sales Transaction v.4a.CSV')

## Exploring  DataFrame

In [57]:
df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [58]:
df.tail()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
536345,C536548,12/1/2018,22168,Organiser Wood Antique White,18.96,-2,12472.0,Germany
536346,C536548,12/1/2018,21218,Red Spotty Biscuit Tin,14.09,-3,12472.0,Germany
536347,C536548,12/1/2018,20957,Porcelain Hanging Bell Small,11.74,-1,12472.0,Germany
536348,C536548,12/1/2018,22580,Advent Calendar Gingham Sack,16.35,-4,12472.0,Germany
536349,C536548,12/1/2018,22767,Triple Photo Frame Cornice,20.45,-2,12472.0,Germany


In [59]:
df.shape

(536350, 8)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


In [61]:
df.describe()

,Price,Quantity,CustomerNo
count,536350.000000,536350.000000,536295.000000
mean,12.662182,9.919347,15227.893178
std,8.490450,216.662300,1716.582932
min,5.130000,-80995.000000,12004.000000
25%,10.990000,1.000000,13807.000000
50%,11.940000,3.000000,15152.000000
75%,14.090000,10.000000,16729.000000
max,660.620000,80995.000000,18287.000000


## Data cleaning

#### Handling missing value

In [62]:
# Check the number of missing values in each column
df.isnull().sum()

TransactionNo     0
Date              0
ProductNo         0
ProductName       0
Price             0
Quantity          0
CustomerNo       55
Country           0
dtype: int64

In [63]:
# Drop rows with missing values and update the DataFrame
df.dropna(inplace=True)

# Print the count of non-null values in each column
print(df.count())

# Verify if there are any missing values remaining
df.isnull().sum()

TransactionNo    536295
Date             536295
ProductNo        536295
ProductName      536295
Price            536295
Quantity         536295
CustomerNo       536295
Country          536295
dtype: int64


TransactionNo    0
Date             0
ProductNo        0
ProductName      0
Price            0
Quantity         0
CustomerNo       0
Country          0
dtype: int64

The CustomerNo column contains 55 missing value ,which is likely represent unidentify customers.To simplify the analysis ,we will drop these unidentify records from the dataset.

#### Removing duplicate values

In [64]:
# Find duplicate rows while keeping the last occurrence
df[df.duplicated(keep='last')]

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
984,581497,12/9/2019,21481,Fawn Blue Hot Water Bottle,7.24,1,17497.0,United Kingdom
1350,581538,12/9/2019,22992,Revolver Wooden Ruler,6.19,1,14446.0,United Kingdom
1354,581538,12/9/2019,22694,Wicker Star,6.19,1,14446.0,United Kingdom
1356,581538,12/9/2019,23275,Set Of 3 Hanging Owls Ollie Beak,6.19,1,14446.0,United Kingdom
1408,581538,12/9/2019,23343,Jumbo Bag Vintage Christmas,6.19,1,14446.0,United Kingdom
...,...,...,...,...,...,...,...,...
535226,536559,12/1/2018,51014L,Feather Pen Light Pink,11.12,12,17873.0,United Kingdom
535305,536569,12/1/2018,22111,Scottie Dog Hot Water Bottle,15.32,1,16274.0,United Kingdom
535323,536569,12/1/2018,21809,Christmas Hanging Tree With Bell,11.53,1,16274.0,United Kingdom
535959,536592,12/1/2018,82613A,Metal Sign Cupcake Single Hook,12.82,1,16592.0,United Kingdom


In [65]:
#removing duplicates
df.drop_duplicates(keep='last')

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom
...,...,...,...,...,...,...,...,...
536345,C536548,12/1/2018,22168,Organiser Wood Antique White,18.96,-2,12472.0,Germany
536346,C536548,12/1/2018,21218,Red Spotty Biscuit Tin,14.09,-3,12472.0,Germany
536347,C536548,12/1/2018,20957,Porcelain Hanging Bell Small,11.74,-1,12472.0,Germany
536348,C536548,12/1/2018,22580,Advent Calendar Gingham Sack,16.35,-4,12472.0,Germany


#### Data type manipulation

In [66]:
# converted float64 to int64
df['CustomerNo'] = pd.to_numeric(df['CustomerNo'], downcast='integer')

# Convert Date Format
df['Date'] = pd.to_datetime(df['Date'])

# Determine the earliest and latest dates
startdate = df['Date'].min()
enddate = df['Date'].max()

# duration between the first and last date
duration = enddate - startdate

# Print the start date, end date, and duration in days
print("Data start date:", startdate)
print("Data end date:", enddate)
print("Duration:", duration.days, "days")


Data start date: 2018-12-01 00:00:00
Data end date: 2019-12-09 00:00:00
Duration: 373 days


Dropping the 2018 records:


The dataset contain inomplete records for the year 2018,as it only includes the data up to the 12 month.Therefore iam dropping the records 2018 to ensure the more effecient and accurate analysis

In [67]:
# Filter the DataFrame to exclude rows where the year is 2018
df = df.loc[df['Date'].dt.year != 2018].copy()
duration = df['Date'].max() - df['Date'].min()

# Print the duration in days
print("Duration:", duration.days, "days")

Duration: 339 days


#### Removing Misinformation Data 

Filter the transaction and canellation to remove the valid order in the corresponding cancellation  

In [68]:
# Filter order records
transactions= df[~df['TransactionNo'].str.contains('C')]
print("Original records:", len(df))
print("Valid orders:", (transactions.shape))

# Filter cancellation records
cancellations=df[df['TransactionNo'].str.contains('C')]
print("Original records:", len(df))
print("Valid cancellation:", (cancellations.shape))


Original records: 494204
Valid orders: (486347, 8)
Original records: 494204
Valid cancellation: (7857, 8)


In [69]:
# Group by 'ProductNo' and 'CustomerNo' and aggregate quantities
grouped = df.groupby(['ProductNo', 'CustomerNo'])['Quantity'].agg(list).reset_index()

# Identify groups with both positive and negative quantities
def has_matching_quantities(quantities):
    # Create a set of absolute values
    abs_quantities = {abs(q) for q in quantities}
    # Check if both positive and negative exist
    return any(q in abs_quantities for q in quantities if q > 0) and any(-q in abs_quantities for q in quantities if q < 0)

# Create a mask for groups to remove
mask_to_remove = grouped[grouped['Quantity'].apply(has_matching_quantities)]

# Identify the indices of valid transactions to remove
indices_to_remove = df[(df['Quantity'] > 0) & df.set_index(['ProductNo', 'CustomerNo']).index.isin(mask_to_remove.set_index(['ProductNo', 'CustomerNo']).index)].index

# Remove the flagged valid transactions 
df = df[~df.index.isin(indices_to_remove)]

# Calculate the number of records removed and total records
records_removed = len(indices_to_remove)  # Count of indices to remove
total_records = df.shape[0] + records_removed  # Total records before removal

print("\nCleaned Dataset (After Removing Flagged Rows):")
print(df)

print(f"\nTotal Records Before Removal: {total_records}")
print(f"Records Removed: {records_removed}")
print(f"Records Remaining: {df.shape[0]}")


Cleaned Dataset (After Removing Flagged Rows):
       TransactionNo       Date ProductNo  \
0             581482 2019-12-09     22485   
1             581475 2019-12-09     22596   
2             581475 2019-12-09     23235   
3             581475 2019-12-09     23272   
4             581475 2019-12-09     23239   
...              ...        ...       ...   
494251       C540030 2019-01-04     22070   
494252       C540097 2019-01-04     22835   
494253       C540097 2019-01-04     22179   
494254       C540097 2019-01-04     22113   
494255       C540109 2019-01-04    85071A   

                                ProductName  Price  Quantity  CustomerNo  \
0             Set Of 2 Wooden Market Crates  21.47        12       17490   
1       Christmas Star Wish List Chalkboard  10.65        36       13069   
2                  Storage Tin Vintage Leaf  11.53        12       13069   
3         Tree T-Light Holder Willie Winkie  10.65        12       13069   
4         Set Of 4 Knick Knack 

"The valid transactions that have corresponding cancellations for the same ProductNo and CustomerNo will be removed, while the cancellations will remain in the dataset. This will make it easier to calculate the upcoming solving hypothesis."

#### Feature Engineering

In [70]:
# Extract from 'Date' column
df['day_name'] = df['Date'].dt.day_name()  # Get the day name (e.g., Monday)
df['day'] = df['Date'].dt.day  # Get the day of the month
df['month'] = df['Date'].dt.month  # Get the month number

# Calculate total sales
df['totalsales'] = df['Price'] * df['Quantity'] 

df.head() 

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,day_name,day,month,totalsales
0,581482,2019-12-09,22485,Set Of 2 Wooden Market Crates,21.47,12,17490,United Kingdom,Monday,9,12,257.64
1,581475,2019-12-09,22596,Christmas Star Wish List Chalkboard,10.65,36,13069,United Kingdom,Monday,9,12,383.40
2,581475,2019-12-09,23235,Storage Tin Vintage Leaf,11.53,12,13069,United Kingdom,Monday,9,12,138.36
3,581475,2019-12-09,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069,United Kingdom,Monday,9,12,127.80
4,581475,2019-12-09,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069,United Kingdom,Monday,9,12,71.64


The dataset was cleaned by removing missing values and duplicates, converting data types, filtering out incomplete 2018 records, and engineering new date and sales features.Now
the dataset is clean, properly formatted, and enriched with additional features, making it ready for exploratory data analysis (EDA).

#### Discriptive Statistics

In [71]:
df.describe(include='all')

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,day_name,day,month,totalsales
count,478302,478302,478302,478302,478302.000000,478302.000000,478302.000000,478302,478302,478302.000000,478302.000000,478302.000000
unique,20932,NaN,3688,3688,NaN,NaN,NaN,37,6,NaN,NaN,NaN
top,573585,NaN,85123A,Cream Hanging Heart T-Light Holder,NaN,NaN,NaN,United Kingdom,Sunday,NaN,NaN,NaN
freq,1111,NaN,2006,2006,NaN,NaN,NaN,433217,91825,NaN,NaN,NaN
mean,NaN,2019-07-22 03:10:24.694857984,NaN,NaN,12.499333,9.238814,15218.746359,NaN,NaN,15.432831,7.195393,104.033395
min,NaN,2019-01-04 00:00:00,NaN,NaN,5.130000,-80995.000000,12004.000000,NaN,NaN,1.000000,1.000000,-840113.800000
25%,NaN,2019-04-26 00:00:00,NaN,NaN,10.920000,1.000000,13810.000000,NaN,NaN,8.000000,4.000000,15.350000
50%,NaN,2019-08-08 00:00:00,NaN,NaN,11.940000,3.000000,15134.000000,NaN,NaN,15.000000,8.000000,42.270000
75%,NaN,2019-10-25 00:00:00,NaN,NaN,14.090000,10.000000,16726.000000,NaN,NaN,23.000000,10.000000,115.300000
max,NaN,2019-12-09 00:00:00,NaN,NaN,660.620000,12540.000000,18287.000000,NaN,NaN,31.000000,12.000000,77622.600000


Describing the cateogorical column iam going to focus on the unique count and top frequency of the categorical features such as,

    *There are 3,688 unique products and 21319 unique transactions recorded in 2019.
    *The products were shipped to 37 different countries.
    *"Cream Hanging Heart T-Light Holder" is the most sold product in the UK.
As for numerical column

    *The cheapest product costs 5.13, while the most expensive costs 660.
    *The highest quantity sold in a single transaction was 80995 units.



## EDA(Exploratory data analysis)

find the transaction and cancellation records

#### Segmentation of Transactions:

In [72]:
# Remove where 'TransactionNo' contains 'C' 
transactions= df[~df['TransactionNo'].str.contains('C')]
print("Original records:", len(df))
print("Valid orders:", (transactions.shape))

Original records: 478302
Valid orders: (470445, 12)


In [73]:
# Select where 'TransactionNo' contains 'C' 
cancellations=df[df['TransactionNo'].str.contains('C')]
print("Original records:", len(df))
print("Valid orders:", (cancellations.shape))

Original records: 478302
Valid orders: (7857, 12)


The original dataset contains  478302 in total.Out of these  transaction contain 470445 and the cancellation 7,857

#### For Total Orders

In [74]:
# Count the number of unique transaction numbers
total_orders = transactions['TransactionNo'].nunique()
total_orders

17850

If a customer orders multiple products at the same time, all those items are usually part of the same order and therefore share the same TransactionNo.so the no of transaction record is 486347 out of these nunique is 18,237

In [ ]:
# Get unique country names
unique_countries = df['Country'].unique()

# total number of unique countries
num_unique_countries = df['Country'].nunique()

# Print the result
print(f"Total number of unique countries: {num_unique_countries}")
print("List of unique countries:")
print(unique_countries)


Total number of unique countries: 37
List of unique countries:
['United Kingdom' 'Norway' 'Belgium' 'Germany' 'France' 'Austria'
 'Netherlands' 'EIRE' 'USA' 'Channel Islands' 'Iceland' 'Portugal' 'Spain'
 'Finland' 'Italy' 'Greece' 'Japan' 'Denmark' 'Sweden' 'Cyprus' 'Malta'
 'Switzerland' 'Australia' 'Czech Republic' 'Poland' 'Hong Kong'
 'Singapore' 'RSA' 'Israel' 'Unspecified' 'United Arab Emirates' 'Canada'
 'European Community' 'Bahrain' 'Brazil' 'Saudi Arabia' 'Lebanon']


The total number of unique countries is 37.

#### Total revenue

In [75]:
# Calculate the total revenue by 'totalsales'
total_revenue=transactions['totalsales'].sum()
total_revenue

np.float64(52246234.69)

Total orders generate a total revenue of approximately 52.25 million.

#### Total quantity

In [76]:
# total sum of the 'Quantity' in transactions
total_quantity=transactions['Quantity'].sum()
total_quantity

np.int64(4672400)

total order generate a total Quantity is 4672400

#### Average Order Value(AOV)

In [77]:
# dividing total revenue by total orders
average_order_value = total_revenue / total_orders
# Print the rounded value 
print(average_order_value.round())

2927.0


The average order value (AOV) is 2927.0, meaning that each order generates an average revenue of 2927.0.

In [78]:
# dividing total quantity by total orders
average_quantity_value=total_quantity/total_orders
# Print the rounded value 
print(average_quantity_value.round())

262.0


Each order generate an average Quantity of 262.

#### Total Revenue in Country

In [ ]:
# Group by Country and sum total sales
grp_country = df.groupby('Country')['totalsales'].sum().reset_index()

# Sort by total sales in descending order and select the top 10
top_10_countries = grp_country.sort_values(by='totalsales', ascending=False).head(10)

print("no of country")

# Display the top 10 countries with the highest total sales
print("Top 10 Countries by Total Sales:")
print(top_10_countries)


Top 10 Countries by Total Sales:
           Country   totalsales
35  United Kingdom  40763064.49
23     Netherlands   2054491.27
10            EIRE   1332558.49
13          France   1182701.46
14         Germany   1145685.73
0        Australia    804987.86
31          Sweden    350958.21
32     Switzerland    339219.70
30           Spain    242876.70
3          Belgium    222861.04




1. **United Kingdom (₹40.76M)** – The UK has the highest sales, much more than any other country.  
2. **Netherlands (₹2.05M) & EIRE (₹1.33M)** – These countries have good sales but are far behind the UK.  
3. **France (₹1.18M) & Germany (₹1.14M)** – Both have similar sales, showing a steady market.  


#### Analysing Product in Demand :

In [79]:
# Group the 'transactions' by 'ProductNo'
product_demand = transactions.groupby('ProductNo').agg({
    'Quantity': 'sum',   # Sum up the total quantity 
    'totalsales': 'sum'  # Sum up the total sales 
}).reset_index()  # Reset the index

# Display the product demand
product_demand  

,ProductNo,Quantity,totalsales
0,10002,609,6780.11
1,10080,303,3227.89
2,10120,177,1810.25
3,10123C,4,43.68
4,10124A,12,128.16
...,...,...,...
3663,90214U,12,73.56
3664,90214V,24,207.21
3665,90214W,16,117.98
3666,90214Y,65,426.63


No of product in the record is 3676.

#### Customer Purchase Frequency :

In [80]:
# Group the 'transactions' by 'CustomerNo' and count the number of unique transactions
customer_purchase_frequency = transactions.groupby('CustomerNo')['TransactionNo'].nunique().reset_index()

# Rename the 'TransactionNo' column to 'PurchaseFrequency' for better readability
customer_purchase_frequency.rename(columns={'TransactionNo': 'PurchaseFrequency'}, inplace=True)
customer_purchase_frequency

,CustomerNo,PurchaseFrequency
0,12004,1
1,12006,1
2,12008,1
3,12024,1
4,12025,1
...,...,...
4544,18280,1
4545,18281,1
4546,18282,2
4547,18283,16


The purchase frequency for each of the 4572 unique customers, providing a clear measure of how many orders each customer has made.

### For Cancellations

In [81]:
# Count the number of unique transaction in cancellations
total_cancellation=cancellations['TransactionNo'].nunique()
# Print the total number of cancellations
print("total cancell:",(total_cancellation))

total cancell: 3082


the order which made with the cancellation record there is a 18k out of 3.08K, unique cancellation

#### Cancellation Count per Product/Customer :


In [82]:
# Group the 'cancellations' by 'ProductNo' and count the number of cancellations 
cancellation_by_product = cancellations.groupby('ProductNo').size().reset_index(name='CancellationCount')

# Group the 'cancellations' by 'CustomerNo' and count the number of cancellations 
cancellation_by_customer = cancellations.groupby('CustomerNo').size().reset_index(name='CancellationCount')

# Print the cancellation counts by product
print(cancellation_by_product)

# Print the cancellation counts by customer
print(cancellation_by_customer)


     ProductNo  CancellationCount
0        10133                  2
1        11001                  4
2        15034                  4
3        15036                  3
4        15039                  3
...        ...                ...
1836    90202D                  1
1837     90204                  1
1838     90208                  1
1839    90209A                  1
1840    90209C                  1

[1841 rows x 2 columns]
      CustomerNo  CancellationCount
0          12346                  1
1          12352                  7
2          12359                  6
3          12362                  8
4          12375                  1
...          ...                ...
1431       18272                  4
1432       18274                 11
1433       18276                  2
1434       18277                  1
1435       18282                  1

[1436 rows x 2 columns]


In the cancellation records, there are 1,436 unique customers and 1,841 unique products, it  counts the cancellations for each group.

#### Cancellation Rate :

calculates the percentage of each product's transactions that were cancelled, providing insight into cancellation trends and potential product issues.








In [83]:
# Merge valid orders and cancellations by ProductNo
product_summary = pd.merge(
    transactions.groupby('ProductNo')['Quantity'].sum().reset_index(),  # Total valid quantity
    cancellations.groupby('ProductNo')['Quantity'].sum().reset_index(),  # Total cancelled quantity
    on='ProductNo', how='outer', suffixes=('_Valid', '_Cancelled')  # Outer join to include all products
).fillna(0)  # Replace NaN with 0

# Calculate cancellation rate
product_summary['CancellationRate'] = product_summary['Quantity_Cancelled'] / (
    product_summary['Quantity_Valid'] + product_summary['Quantity_Cancelled']
)

# Display cancellation rate
product_summary['CancellationRate']


0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3683    0.0
3684    0.0
3685    0.0
3686    0.0
3687    0.0
Name: CancellationRate, Length: 3688, dtype: float64

Top 10 product by revenue:

In [84]:
# Calculate total revenue from transactions
total_revenue = transactions['totalsales'].sum()

# Group by ProductName to calculate revenue per product
valid_revenue = transactions.groupby('ProductName').agg({
    'Quantity': 'sum',
    'totalsales': 'sum'
}).reset_index()

# Sort products by total revenue in descending order and select the top 10
top_products_by_revenue = valid_revenue.sort_values(by='totalsales', ascending=False).head(10)

print(top_products_by_revenue[['Quantity', 'totalsales']].mean())
# Display the top 10 products by revenue
top_products_by_revenue


Quantity       29489.200
totalsales    328444.325
dtype: float64


,ProductName,Quantity,totalsales
2421,Popcorn Holder,43885,452195.47
3588,World War 2 Gliders Asstd Designs,42796,441487.44
202,Assorted Colour Bird Ornament,33594,386668.10
806,Cream Hanging Heart T-Light Holder,27657,351991.94
2111,Pack Of 72 Retrospot Cake Cases,32110,343423.22
2478,Rabbit Night Light,29799,317914.55
2176,Party Bunting,16663,262286.23
210,Assorted Colours Silk Fan,22333,245190.80
2078,Pack Of 12 London Tissues,23405,242255.23
2109,Pack Of 60 Pink Paisley Cake Cases,22650,241030.27


The average quantity sold per product is 29,489.2 units, while the average total sales revenue per product is ₹328,444.33.



In [85]:
# Create a subplot with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for Total Revenue (color based on revenue)
fig.add_trace(
    go.Bar(
        x=top_products_by_revenue['ProductName'],
        y=top_products_by_revenue['totalsales'],
        name="Total Revenue",
        marker=dict(color=top_products_by_revenue['totalsales'], colorscale='Blues'),  # Color scale based on revenue
        text=top_products_by_revenue['totalsales'],
        textposition='outside'
    ),
    secondary_y=False
)

# Add line chart for Quantity Trend
fig.add_trace(
    go.Scatter(
        x=top_products_by_revenue['ProductName'],
        y=top_products_by_revenue['Quantity'],
        mode='lines+markers',
        name="Quantity Sold",
        marker=dict(size=8),
        line=dict(width=3)
    ),
    secondary_y=True
)

## Improve layout
fig.update_layout(
    title="Top 10 Products by Revenue with Quantity Trend",
    xaxis_title="Product Name",
    width=1000,
    height=600,
    template="plotly_dark",  # Dark theme applied
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Set y-axis labels
fig.update_yaxes(title_text="Total Revenue", secondary_y=False)
fig.update_yaxes(title_text="Quantity Sold", secondary_y=True)

# Show the chart
fig.show()



The analysis shows strong total revenue among the top products. The Popcorn Holder and World War 2 Gliders Assorted Designs generate the highest revenue, while the Assorted Colours Silk Fan, Pack of 12 London Tissues, and Pack of 60 Pink Paisley Cake Cases contribute the least revenue. Additionally, homeware generates higher revenue compared to adult gifts.

Monthly Revenue Trend

In [86]:
# Calculate monthly valid and canceled revenue
monthly_valid_revenue = transactions.groupby('month')['totalsales'].sum().reset_index().rename(columns={'totalsales': 'Total_Revenue'})
monthly_canceled_revenue = cancellations.groupby('month')['totalsales'].sum().reset_index().rename(columns={'totalsales': 'Cancelled_Revenue'})

# Merge both dataframes
monthly_revenue = pd.merge(
    monthly_valid_revenue, 
    monthly_canceled_revenue, 
    on='month', 
    how='outer'
).fillna(0)

# Plot Total Revenue vs. Cancelled Revenue
fig = px.line(
    monthly_revenue, 
    x='month', 
    y=['Total_Revenue', 'Cancelled_Revenue'], 
    title='Monthly Revenue Trend (Total vs. Cancelled)',
    markers=True, 
    color_discrete_sequence=px.colors.qualitative.Dark24,  # Darker color scheme
    template="plotly_dark"  # Dark theme applied
)

# Update layout for better visualization
fig.update_traces(line=dict(width=3, dash="dot"), marker=dict(size=10))
fig.update_layout(
    xaxis_title="Month", 
    yaxis_title="Revenue", 
    title_font=dict(size=20),  
    font=dict(size=14),
    legend_title="Revenue Type"
)

fig.show()



The monthly trend chart indicates two types of trend lines: transactions, cancellations. The total sales cancellations are relatively lower compared to transactions. The highest total sales were recorded in November (7.38M), followed by October (6.5M) and September (6.2M). However, December sales dropped significantly to 1.44M, likely due to early Christmas purchases made in october and November.

####  Monthly revenue trend By Product Name

In [87]:
# Group by ProductName and calculate total revenue
total_revenue_per_product = transactions.groupby('ProductName')['totalsales'].sum().reset_index()

# Get the top 10 products by total revenue using head(10)
top_10_products = total_revenue_per_product.sort_values(by='totalsales', ascending=False).head(10)['ProductName']

# Filter the original monthly revenue data for these top 10 products
monthly_product_revenue = transactions.groupby(['month', 'ProductName'])['totalsales'].sum().reset_index()
top_10_monthly_revenue = monthly_product_revenue[monthly_product_revenue['ProductName'].isin(top_10_products)]

#  Plot the trend line for top 10 products
fig = px.line(
    top_10_monthly_revenue, 
    x='month', 
    y='totalsales', 
    color='ProductName',  # Different color for each product
    title='Monthly Total Revenue Trend (Top 10 Products)',
    markers=True, 
    template="plotly_dark"  # Dark theme for better visualization
)

# Update layout for better visualization
fig.update_traces(line=dict(width=2), marker=dict(size=6))
fig.update_layout(
    xaxis_title="Month", 
    yaxis_title="Total Revenue", 
    font=dict(size=14),
    legend_title="Product Name"
)

# Show the chart
fig.show()


In November, the Rabbit Night Light saw high sales due to winter and home decor demand. The Popcorn Holder peaked in May and November, coinciding with summer and holidays as people hosted movie nights. The Assorted Color Bird Ornament experienced a sales surge in August, linked to Caribbean culture celebrations. The World War 2 Gliders Assorted Design saw increased sales in April, around anniversaries and remembrance days for historical events.

#### Top 10 Customers by Purchase Frequency

In [88]:
# Calculate customer purchase frequency
customer_purchase_frequency = transactions.groupby('CustomerNo')['TransactionNo'].count().reset_index()
customer_purchase_frequency.rename(columns={'TransactionNo': 'PurchaseFrequency'}, inplace=True)

# Get the top 10 customers by purchase frequency
top_customers_by_frequency = customer_purchase_frequency.sort_values(by='PurchaseFrequency', ascending=False).head(10)

# Convert CustomerNo to string for better labeling
top_customers_by_frequency['CustomerNo'] = top_customers_by_frequency['CustomerNo'].astype(str)

# Create a colorful bar chart
fig = px.bar(
    top_customers_by_frequency,
    x='CustomerNo',
    y='PurchaseFrequency',
    text='PurchaseFrequency',  # Display frequency values
    title="Top 10 Customers by Purchase Frequency",
    labels={'CustomerNo': 'Customer Number', 'PurchaseFrequency': 'Purchase Frequency'},
    color='PurchaseFrequency',  # Color based on purchase frequency
    template="plotly_dark"  # Dark theme
)

# Update layout for better visualization
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    xaxis_title="Customer Number",
    yaxis_title="Purchase Frequency",
    xaxis_tickangle=-45,
    font=dict(size=14),
    width=900,
    height=600,
    title_font=dict(size=20)
)

# Show the plot
fig.show()


The customer purchase frequency reflects the number of transactions made by each customer. The highest recorded frequency is 6222, indicating regular and loyal customers. However, customers with low frequency might be bulk purchasers.

#### Top 10 Customers by Total Spending 

In [89]:
# Calculate total spending per customer
customer_spending = transactions.groupby('CustomerNo')['totalsales'].sum().reset_index()
customer_spending.rename(columns={'totalsales': 'TotalSpending'}, inplace=True)

# Get the top 10 customers by total spending
top_customers_by_spending = customer_spending.sort_values(by='TotalSpending', ascending=False).head(10)

# Convert CustomerNo to string for better labeling
top_customers_by_spending['CustomerNo'] = top_customers_by_spending['CustomerNo'].astype(str)

# Create a bar chart
fig = px.bar(
    top_customers_by_spending,
    x='CustomerNo',
    y='TotalSpending',
    text='TotalSpending',  # Display spending values
    title="Top 10 Customers by Total Spending",
    labels={'CustomerNo': 'Customer Number', 'TotalSpending': 'Total Spending'},
    color='TotalSpending',  # Color based on total spending
    template="plotly_dark"  # Dark theme
)

# Update layout for better visualization
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title="Customer Number",
    yaxis_title="Total Spending",
    xaxis_tickangle=-45,
    font=dict(size=14),
    width=900,
    height=600,
    title_font=dict(size=20)
)

# Show the plot
fig.show()


Customer total spending indicates bulk purchases, reflecting customers who make fewer transactions but buy larger quantities in a single transaction.

#### Top 10 Products by Cancellation Count :

In [90]:
# Group by ProductName to count cancellations
cancellation_by_product = cancellations.groupby('ProductName').size().reset_index(name='CancellationCount')

# Get the top 10 products with the highest cancellations
top_products_by_cancellations = cancellation_by_product.sort_values(by='CancellationCount', ascending=False).head(10)

# Create a bar chart with a red color theme
fig = px.bar(
    top_products_by_cancellations,
    x='ProductName',
    y='CancellationCount',
    text='CancellationCount',  # Display cancellation values
    title="Top 10 Products by Cancellation Count",
    labels={'ProductName': 'Product Name', 'CancellationCount': 'Cancellation Count'},
    color_discrete_sequence=['#B22222'],  # Deep red color for cancellations
    template="plotly_dark"  # Dark theme for contrast
)

# Update layout for better visualization
fig.update_traces(
    texttemplate='%{text}', 
    textposition='outside'
)

fig.update_layout(
    xaxis_title="Product Name",
    yaxis_title="Cancellation Count",
    xaxis_tickangle=-45,
    width=900,
    height=600,
    title_font=dict(size=20),
    plot_bgcolor='black',  # Ensure dark background
    paper_bgcolor='black'
)

# Show the plot
fig.show()


The top  products are highly in demand due to their popularity, leading to stock shortages.

#### Top 10 Product stock to be stored in warehouse

In [91]:
# Aggregate monthly cancellations by product
cancellation_by_product_monthly = cancellations.groupby(['month', 'ProductName']).size().reset_index(name='CancellationCount')

# Get the top 10 products with the highest total cancellations (alternate to nlargest)
top_10_products = cancellation_by_product_monthly.groupby('ProductName')['CancellationCount'].sum().sort_values(ascending=False).head(10).index

# Filter data for only the top 10 products
top_products_monthly = cancellation_by_product_monthly[cancellation_by_product_monthly['ProductName'].isin(top_10_products)]

# Create a line chart using Plotly to show the trend over months
fig = px.line(
    top_products_monthly,
    x='month',
    y='CancellationCount',
    color='ProductName',
    title="Top 10 Products - Monthly Cancellation Trend",
    labels={'month': 'Month', 'CancellationCount': 'Cancellation Count', 'ProductName': 'Product Name'},
    color_discrete_sequence=px.colors.qualitative.Dark24,  # Dark color theme for better differentiation
    markers=True  # Show markers for each data point
)

# Update layout for better visualization
fig.update_traces(line=dict(width=3, shape='spline'))  # Smooth curve
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Cancellation Count",
    xaxis_tickangle=0,
    template="plotly_dark",  # Dark theme
    title_font=dict(size=20, color='red'),
    legend_title="Product Name",
    width=1000,
    height=600
)

# Show the plot
fig.show()


 Regency Cakstand 3 Tier gives highly cancelled in every month.The cancellation analysis reveals that homeware products experience high cancellation rates every month.However, from October to November, cancellations peak as these months coincide with festive celebrations. During this pre-holiday period, people tend to stock up on homeware products in preparation for upcoming festivities, leading to a surge in demand and, consequently, an increase in cancellations due to stock shortages or fulfillment issues.

#### Top 10 Customers by Cancellation Count :

In [92]:
# Group cancellations by customer
cancellation_by_customer = cancellations.groupby('CustomerNo')['TransactionNo'].count().reset_index(name='CancellationCount')

# Get the top 10 customers with the highest cancellations
top_customers_by_cancellations = cancellation_by_customer.sort_values(by='CancellationCount', ascending=False).head(10)

# Convert CustomerNo to string for better labeling
top_customers_by_cancellations['CustomerNo'] = top_customers_by_cancellations['CustomerNo'].astype(str)

# Create a dark-themed bar chart for cancellation counts
fig = px.bar(
    top_customers_by_cancellations,
    x='CustomerNo',
    y='CancellationCount',
    text='CancellationCount',  # Display count values
    title="Top 10 Customers by Cancellation Count",
    labels={'CustomerNo': 'Customer Number', 'CancellationCount': 'Cancellation Count'},
    color_discrete_sequence=['#B22222'],  # Bright red for contrast
    template="plotly_dark"  # Dark theme
)

# Update layout for better visualization
fig.update_traces(
    texttemplate='%{text}', 
    textposition='outside'
)

fig.update_layout(
    xaxis_title="Customer Number",
    yaxis_title="Cancellation Count",
    xaxis_tickangle=-45,
    width=900,
    height=600,
    title_font=dict(size=22, color='white'),
    font=dict(size=14, color='white'),
    plot_bgcolor='black',  # Dark background
    paper_bgcolor='black'
)

# Show the plot
fig.show()


The cancellation count by customer indicates that cancellations may be due to products being out of stock.

#### Groupby Country:

In [134]:
# Group by Country and sum total sales
grp_country = transactions.groupby('Country')['totalsales'].sum().reset_index()

# Create a Choropleth map with a dark background
fig = px.choropleth(
    grp_country, 
    locations='Country',  
    locationmode='country names',  
    color='totalsales',  
    hover_name='Country',  
    color_continuous_scale='plasma',  # Bright, high-contrast scale
    title='Total Sales by Country'
)

# Customize the layout with a dark background
fig.update_layout(
    geo=dict(
        showcoastlines=False,  
        projection_type="natural earth",  
        bgcolor='black',  
        showland=True,  
        landcolor='dimgray'
    ),
    paper_bgcolor='black',  
    plot_bgcolor='black'
)

# Show the interactive map
fig.show()


The United Kingdom generates the highest revenue, exceeding ₹40 million.

#### Cancellation Rate vs. Product Demand

In [ ]:

# Merge valid orders and cancellations
product_summary = pd.merge(
    transactions.groupby('ProductNo')['Quantity'].sum().reset_index(),
    cancellations.groupby('ProductNo')['Quantity'].sum().reset_index(),
    on='ProductNo', how='outer', suffixes=('_Valid', '_Cancelled')
).fillna(0)

# Calculate Cancellation Rate
product_summary['CancellationRate'] = product_summary['Quantity_Cancelled'] / (
    product_summary['Quantity_Valid'] + product_summary['Quantity_Cancelled']
)

# Create an interactive scatter plot
fig = px.scatter(
    product_summary,
    x='Quantity_Valid',
    y='CancellationRate',
    title="Cancellation Rate vs. Product Demand",
    labels={'Quantity_Valid': 'Product Demand (Quantity)', 'CancellationRate': 'Product Rate'},
    color='CancellationRate',  # Color based on cancellation rate
    color_continuous_scale='oranges',  # Orange theme
    template='plotly_dark'  # Dark theme for better contrast
)

# Update layout
fig.update_layout(
    width=950,
    height=600,
    title_font=dict(size=22, color='white'),
    font=dict(size=14, color='white'),
    plot_bgcolor='black',
    paper_bgcolor='black'
)

# Show the plot
fig.show()


The x-axis represents product demand (total valid orders), while the y-axis shows the cancellation rate (canceled quantity vs. total orders). Each dot represents a product—its position reflects popularity (x) and cancellation proportion (y). High-demand products with low cancellations indicate strong performance and reliability.

## Conclusion:

### Key Performance Metrics
### Transaction Overview
* Total Records: 478,302
* Valid Transactions: 470,445 (98.4%)  
* Cancellations: 7,857 (1.6%)
* Unique Orders: 18,237
* Total Revenue: ₹52.25M
* Average Order Value (AOV): ₹2,927
* Average Quantity per Order: 262 units

### Geographic Reach
* Active Markets: 37 countries
* Market Leader: United Kingdom (₹40.76M revenue)
* Key Secondary Markets:
   * Netherlands (₹2.05M)
   * EIRE (₹1.33M) 
   * France (₹1.18M)
   * Germany (₹1.14M)

## Critical Insights

### Product Performance

#### 1. Product Portfolio
* Total Active Products: 3,676
* Average Revenue per Product: ₹328,444
* Average Quantity Sold: 29,489 units

#### 2. Customer Base  
* Unique Customers: 4,572
* Active Buyers: Varied purchase frequencies
* High-Value Customer Segment: Identified bulk purchasers

### Seasonal Trends

#### 1. Peak Performance
* Highest Revenue: November (₹7.38M)
* Strong Months: October (₹6.5M), September (₹6.2M)
* Notable Decline: December (₹1.44M)

#### 2. Product-Specific Trends
* Rabbit Night Light: Winter peak
* Popcorn Holder: Bi-modal peak (May, November)
* Assorted Color Bird Ornament: August surge
* World War 2 Gliders: April spike

### Cancellation Analysis

#### 1. Overview
* Unique Cancellations: 3,080
* Affected Products: 1,841
* Impacted Customers: 1,436

#### 2. Pattern Recognition
* Higher cancellation rates in homeware category
* Peak cancellations: October-November
* Strong correlation with festive seasons
* Notable product: Regency Cakstand 3 Tier (consistent cancellations)

## Strategic Recommendations

### Inventory Management

#### 1. Stock Optimization
* Implement predictive stocking for high-demand periods
* Increase buffer stock for frequently cancelled items
* Develop season-specific inventory strategies

#### 2. Demand Planning
* Factor in historical cancellation patterns
* Adjust stock levels for seasonal peaks
* Create product-specific stocking strategies

### Market Development

#### 1. Geographic Expansion
* Focus on strengthening secondary markets
* Develop market-specific product assortments
* Implement targeted marketing strategies

#### 2. Customer Engagement
* Develop loyalty programs for high-frequency buyers
* Create specialized offerings for bulk purchasers
* Implement proactive communication for potential cancellations

### Operational Excellence

#### 1. Supply Chain
* Review supplier capacity for high-demand products
* Implement better forecasting for seasonal items
* Develop contingency plans for peak periods

#### 2. Customer Service
* Enhanced communication during festive seasons
* Proactive stock updates for popular items
* Dedicated support for high-value customers

## Future Focus Areas
1. Market penetration strategies for secondary markets
2. Development of seasonal product portfolios
3. Enhanced inventory management systems
4. Customer retention and loyalty programs
5. Cancellation reduction strategies

---
*This analysis provides a comprehensive foundation for data-driven decision-making and strategic planning. Regular monitoring and updates of these metrics will ensure continued optimization of business operations and customer satisfaction.*